In [ ]:
from helikite import Cleaner, instruments
import os
import datetime

cleaner = Cleaner(
    instruments=[
        instruments.flight_computer_v1,
        instruments.smart_tether,
        instruments.pops,
        instruments.msems_readings,
        instruments.msems_inverted,
        instruments.msems_scan,
        instruments.stap,
    ], 
    reference_instrument=instruments.flight_computer_v1,
    input_folder=os.path.join(os.getcwd(), "rawdata"),
    flight_date=datetime.date(2024,4,2),
    time_takeoff=datetime.datetime(2024,4,2,10,0,35),
    time_landing=datetime.datetime(2024,4,2,13,4,4),
    time_offset=datetime.time(0),
)

cleaner.set_time_as_index()
cleaner.data_corrections()
cleaner.set_pressure_column()
# cleaner.correct_time_and_pressure(max_lag=180)
cleaner.remove_duplicates()
cleaner.merge_instruments()
cleaner.export_data()

In [ ]:
from helikite.metadata.utils import load_parquet

df, metadata = load_parquet("level0_2024-04-02T09-45.parquet")

In [ ]:
print(metadata)